In [3]:
# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
# FILL IN THESE PATHS
GCTA="/cellar/users/jlz014/fusion_twas-master/gcta_nr_robust"
PLINK="<PATH TO PLINK>"
GEMMA="/cellar/users/jlz014/gemma/bin/gemma"
# ALTERNATIVELY: ENSURE THAT plink, gcta, gemma CAN BE CALLED FROM PATH AND REMOVE --PATH_* FLAGS BELOW

# PATH TO DIRECTORY CONTAINING IMPUTED TCGA GENOTYPES
TCGA_GENO='/cellar/users/jlz014/Data/TCGA/TCGA_EUR_imputed' 
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY

In [4]:
# PATH TO GEUVADIS GENE EXPRESSION MATRIX:
PRE_GEXP_DIR='/data/cellardata/users/jlz014/TCGA/expression/'
PRE_GEXP="TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt"

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
# test white population first
PRE_GENO="EUR_imputed_females_filtered"

# PATH TO OUTPUT DIRECTORY (population-specific subdirs will be made)
OUT_DIR="./TEST_WEIGHTS"

# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
if [[ ! -e $OUT_DIR ]]; then
    mkdir -p $OUT_DIR
fi

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=10

# make *.ID for gene expression file
echo `awk 'NR==1' $PRE_GEXP_DIR$PRE_GEXP` | tr ' ' '\n' | tail -n+5 > $PRE_GEXP.ID.tmp
paste $PRE_GEXP.ID.tmp $PRE_GEXP.ID.tmp | column -s $'\t' -t > $PRE_GEXP.ID
rm $PRE_GEXP.ID.tmp

In [5]:
# make *.ID for gene expression file before script is submitted with the following commands:
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY
PRE_GEXP_DIR='/data/cellardata/users/jlz014/TCGA/expression/'
PRE_GEXP="TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt"
echo `awk 'NR==1' $PRE_GEXP_DIR/$PRE_GEXP` | tr ' ' '\n' | tail -n+5 > $PRE_GEXP.ID.tmp
paste $PRE_GEXP.ID.tmp $PRE_GEXP.ID.tmp | column -s $'\t' -t > $PRE_GEXP.ID
rm $PRE_GEXP.ID.tmp

In [2]:
head $PRE_GEXP.ID

TCGA-A8-A09Z  TCGA-A8-A09Z
TCGA-5L-AAT1  TCGA-5L-AAT1
TCGA-LL-A73Z  TCGA-LL-A73Z
TCGA-AR-A2LQ  TCGA-AR-A2LQ
TCGA-C8-A12X  TCGA-C8-A12X
TCGA-A8-A081  TCGA-A8-A081
TCGA-BH-A0W7  TCGA-BH-A0W7
TCGA-S3-AA14  TCGA-S3-AA14
TCGA-A1-A0SQ  TCGA-A1-A0SQ
TCGA-W8-A86G  TCGA-W8-A86G


In [6]:
# TEST ONE 
# --- BEGIN SCRIPT:

NR="${BATCH_START}_${BATCH_END}"
mkdir --parents tmp/$NR
mkdir --parents hsq/$NR
mkdir --parents out/$NR

# test first gene in $PRE_GEXP
PARAM=`awk 'NR==2' $PRE_GEXP_DIR$PRE_GEXP`
echo $PARAM
# Get the gene positions +/- 500kb (1MB window)
CHR=`echo $PARAM | awk '{ print $2 }'`
P0=`echo $PARAM | awk '{ print $3 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

# Pull out the current gene expression phenotype
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste $PRE_GEXP.ID - > $OUT.pheno

echo 'running plink'
# # Get the locus genotypes for all samples and set current gene expression as the phenotype
plink2 --bfile $TCGA_GENO/$PRE_GENO.$CHR --pheno $OUT.pheno --make-bed --out $OUT --keep $OUT.pheno --chr $CHR --from-bp $P0 --to-bp $P1 --extract $TCGA_GENO/$PRE_GENO.$CHR.bim

# echo 'Process all samples together (for reference purposes only since this is multi-ethnic data)'
# # Process all samples together (for reference purposes only since this is multi-ethnic data)
# mkdir $OUT_DIR/ALL
# FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
# echo $FINAL_OUT

# Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.ALL.tmp --out $FINAL_OUT --verbose 1 --hsq_set 0.5 --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,top1,enet

# ALTERNATIVELY ADD COVARIATES HERE USING THE --covar FLAG
# MINIMAL COMMAND IS: `Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT`

# echo 'Append heritability output to hsq file'
# # Append heritability output to hsq file
# head $FINAL_OUT.hsq
# cat $FINAL_OUT.hsq >> hsq/$NR.hsq

# echo 'clean up'
# # Clean-up just in case
# rm -f $FINAL_OUT.hsq $OUT.tmp.*

# echo 'remove all intermediate files'
# # Remove all intermediate files
# rm $OUT.*

echo 'done'

ENSG00000000419 20 49552684 49575069 -0.741658478192737 -0.0952878122051453 0.561424822441768 -1.30983090425029 -0.2322969207296 1.1635385691527 0.261360606434587 -1.21598010110204 1.71057139968799 -0.495239437590013 -0.75877860749 -0.124832641932182 -1.1635385691527 0.963054547438794 -0.22506250846222 0.380138579525698 -0.910233924905761 -2.11211410413361 -0.811537320762164 0.625875754409179 -0.868369424593738 -1.48414880852707 0.803400639985522 -1.04470907170884 0.0917485226718669 -0.732411362241843 -0.103550876090444 -2.23672082094666 -0.0587645509836792 -2.26634647382571 0.821372843922416 -0.16043320914408 1.55100820015369 0.131938669358162 1.90201162764356 0.438904105849384 0.0988282957743533 0.470163722234286 1.14524250362602 0.32265824618159 -0.18666758263932 -0.229884117579232 -0.572460100557655 -0.222653671572159 1.00693838147744 2.82785957952873 0.278415355587258 0.831288474199385 0.813171106895486 -2.7183696551449 -0.788886974508346 1.32663393933315 -0.258930522864627 -0.453

ENSG00000000419 20 49052684 50075069
running plink
PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.log.
Options in effect:
  --bfile /cellar/users/jlz014/Data/TCGA/TCGA_EUR_imputed/EUR_imputed_females_filtered.20
  --chr 20
  --extract /cellar/users/jlz014/Data/TCGA/TCGA_EUR_imputed/EUR_imputed_females_filtered.20.bim
  --from-bp 49052684
  --keep tmp/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.pheno
  --make-bed
  --out tmp/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419
  --pheno tmp/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.pheno
  --to-bp 50075069

249894 MB RAM detected; reserving 124947 MB for main workspace.
2842 variants loaded from .bim file.
3799 people (0 males, 3799 females) loaded fr

In [11]:
# --- BEGIN SCRIPT:

NR="${BATCH_START}_${BATCH_END}"
mkdir --parents tmp/$NR
mkdir --parents hsq/$NR
mkdir --parents out/$NR
# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
mkdir $OUT_DIR

# Loop through each gene expression phenotype in the batch
cat $PRE_GEXP_DIR$PRE_GEXP | awk -vs=$BATCH_START -ve=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do

# Get the gene positions +/- 500kb (1MB window)
CHR=`echo $PARAM | awk '{ print $2 }'`
P0=`echo $PARAM | awk '{ print $3 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1

# Pull out the current gene expression phenotype
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste $PRE_GEXP.ID - > $OUT.pheno

echo 'running plink'
# Get the locus genotypes for all samples and set current gene expression as the phenotype
plink2 --bfile $TCGA_GENO/$PRE_GENO.$CHR --pheno $OUT.pheno --make-bed --out $OUT --keep $OUT.pheno --chr $CHR --from-bp $P0 --to-bp $P1 --extract $TCGA_GENO/$PRE_GENO.$CHR.bim

echo 'Process all samples together (for reference purposes only since this is multi-ethnic data)'
# Process all samples together (for reference purposes only since this is multi-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
echo $FINAL_OUT

Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.ALL.tmp --out $FINAL_OUT --verbose 0 --hsq_set 0.098 --PATH_gcta $GCTA --PATH_gemma $GEMMA --models top1,enet --noclean

# ALTERNATIVELY ADD COVARIATES HERE USING THE --covar FLAG
# MINIMAL COMMAND IS: `Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT`

# echo 'Append heritability output to hsq file'
# # Append heritability output to hsq file
# cat $FINAL_OUT.hsq >> hsq/$NR.hsq

# echo 'clean up'
# # Clean-up just in case
# rm -f $FINAL_OUT.hsq $OUT.tmp.*

# echo 'remove all intermediate files'
# # Remove all intermediate files
# rm $OUT.*

echo 'done'
done

mkdir: cannot create directory ‘./TEST_WEIGHTS’: File exists
ENSG00000000419 20 49052684 50075069
running plink
PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.log.
Options in effect:
  --bfile /cellar/users/jlz014/Data/TCGA/imputed/split_chr/EUR_imputed_females_filtered.20
  --chr 20
  --extract /cellar/users/jlz014/Data/TCGA/imputed/split_chr/EUR_imputed_females_filtered.20.bim
  --from-bp 49052684
  --keep tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.pheno
  --make-bed
  --out tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419
  --pheno tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000419.pheno
  --to-bp 50075069

249894 MB RAM detected; reserving 124947 MB for main workspace.

--extract: 1695 variants remaining.
--keep: 564 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 564 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
1695 variants and 564 people pass filters and QC.
Phenotype data is quantitative.
--make-bed to
tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000938.bed
+
tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000938.bim
+
tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000000938.fam
... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Process all samples together (fo

[1] "./TEST_WEIGHTS/ALL/ALL.ENSG00000001084.wgt.RDat"
[1] "output saved to ./TEST_WEIGHTS/ALL/ALL.ENSG00000001084.wgt.RDat"
done
ENSG00000001167 6 40540721 41565689
running plink
PLINK v1.90p 64-bit (25 Mar 2016)          https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000001167.log.
Options in effect:
  --bfile /cellar/users/jlz014/Data/TCGA/imputed/split_chr/EUR_imputed_females_filtered.6
  --chr 6
  --extract /cellar/users/jlz014/Data/TCGA/imputed/split_chr/EUR_imputed_females_filtered.6.bim
  --from-bp 40540721
  --keep tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000001167.pheno
  --make-bed
  --out tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000001167
  --pheno tmp/1_10/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt.ENSG00000001167.pheno
  --to-bp 41565689


In [7]:
pwd

/cellar/users/jlz014/fusion_twas-master
